# Necessary Installations

In [ ]:
! pip install wandb

---

# Cloning and Switching to the `additional_training` Branch in Git

In [ ]:
%cd /kaggle/working

In [ ]:
! git clone https://github.com/prathamadh/Texture_training.git

In [ ]:
%cd /kaggle/working/Texture_training

In [ ]:
! git fetch origin
! git checkout additional_training

In [ ]:
! git branch

---

# Importing Required Libraries

In [9]:
import os
import torch
import wandb
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, HfFolder, Repository, create_repo, upload_file
from huggingface_hub import login
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model
from huggingface_hub import hf_hub_download

---

# Model Setup and Loading Checkpoints

In [10]:
class DeconvNet(nn.Module):
    def __init__(self, in_channels, target_height, target_width):
        super(DeconvNet, self).__init__()
        self.target_height = target_height
        self.target_width = target_width

        # Deconvolutional layers
        self.deconv1 = nn.ConvTranspose2d(in_channels, 512, kernel_size=4, stride=2, padding=1)  # (2x upsample)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1)          # (2x upsample)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)          # (2x upsample)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)           # (2x upsample)
        self.deconv5 = nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1)             # (2x upsample)

    def forward(self, x):
        x = self.deconv1(x)
        x = torch.relu(x)

        x = self.deconv2(x)
        x = torch.relu(x)

        x = self.deconv3(x)
        x = torch.relu(x)

        x = self.deconv4(x)
        x = torch.relu(x)

        x = self.deconv5(x)

        # Resize to match the target dimensions
        x = nn.functional.interpolate(x, size=(self.target_height, self.target_width), mode='bilinear', align_corners=False)

        return x

In [11]:
class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch

    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)),
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf
    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, out_chans=1,
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0.,
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 **kwargs,):

        super().__init__()



        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)



        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j],
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]




        # Output head for roughness map prediction
        self.deconv = DeconvNet(in_channels= dims[-1], target_height=600, target_width=600)


        self.apply(self._init_weights)


    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        features = []
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
            features.append(x)

        x = self.deconv(x)
        features.append(x)

        return features
        # return features # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        #x = self.forward_features(x)
        #x = self.head(x)
        features = self.forward_features(x)

        return features

class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape, )

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]

            return x

model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
    "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    "convnext_small_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth",
    "convnext_base_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth",
    "convnext_large_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth",
    "convnext_xlarge_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth",
}

def convnext_tiny(pretrained=True,in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        checkpoint = torch.load(kwargs['checkpoint'], map_location="cpu")
        # url = model_urls['convnext_tiny_22k'] if in_22k else model_urls['convnext_tiny_1k']
        # checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model_dict = model.state_dict()
        pretrained_dict = {}
        unmatched_pretrained_dict = {}
        for k, v in checkpoint['model'].items():
            if k in model_dict:
                pretrained_dict[k] = v
            else:
                unmatched_pretrained_dict[k] = v
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)
        print(
            'Successfully loaded pretrained %d paras, and %d paras are unmatched.'
            %(len(pretrained_dict.keys()), len(unmatched_pretrained_dict.keys())))
        print('Unmatched pretrained paras are:', unmatched_pretrained_dict.keys())
    return model

def convnext_small(pretrained=True,in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        checkpoint = torch.load(kwargs['checkpoint'], map_location="cpu")
        # url = model_urls['convnext_small_22k'] if in_22k else model_urls['convnext_small_1k']
        # checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model_dict = model.state_dict()
        pretrained_dict = {}
        unmatched_pretrained_dict = {}
        for k, v in checkpoint['model'].items():
            if k in model_dict:
                pretrained_dict[k] = v
            else:
                unmatched_pretrained_dict[k] = v
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)
        print(
            'Successfully loaded pretrained %d paras, and %d paras are unmatched.'
            %(len(pretrained_dict.keys()), len(unmatched_pretrained_dict.keys())))
        print('Unmatched pretrained paras are:', unmatched_pretrained_dict.keys())
    return model

def convnext_base(pretrained=True, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], **kwargs)
    if pretrained:
        # checkpoint = torch.load(kwargs['checkpoint'], map_location="cpu")
        url = model_urls['convnext_base_22k'] if in_22k else model_urls['convnext_base_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model_dict = model.state_dict()

        # print(model_dict)
        # print(checkpoint)

        pretrained_dict = {}
        unmatched_pretrained_dict = {}

        for k, v in checkpoint['model'].items():
            if k in model_dict.keys():
                pretrained_dict[k] = v
            else:
                unmatched_pretrained_dict[k] = v

        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

        print(f'The type of checkpoint is {type(checkpoint)}.')
        print(f'The type of model.state_dict is {type(model.state_dict)}.')
        print(f'The type of model.state_dict() is {type(model.state_dict())}.')

        for name,param in model.named_parameters():
          if name in pretrained_dict.keys():
              param.requires_grad = False
          else :
              param.requires_grad = True




        print('\n')

        print(f'The keys in pretrained_dict are : \n {pretrained_dict.keys()}')

        print('\n')

        print(f'The keys in unmatched_pretrained_dict are : \n {unmatched_pretrained_dict.keys()}')

        print('\n')

        print(
            'Successfully loaded pretrained %d paras, and %d paras are unmatched.'
            %(len(pretrained_dict.keys()), len(unmatched_pretrained_dict.keys())))

        print('\n')

        print('Unmatched pretrained paras are:', unmatched_pretrained_dict.keys())



    return model

def convnext_large(pretrained=True, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
    if pretrained:
        # checkpoint = torch.load(kwargs['checkpoint'], map_location="cpu")
        url = model_urls['convnext_large_22k'] if in_22k else model_urls['convnext_large_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model_dict = model.state_dict()
        pretrained_dict = {}
        unmatched_pretrained_dict = {}
        for k, v in checkpoint['model'].items():
            if k in model_dict:
                pretrained_dict[k] = v
            else:
                unmatched_pretrained_dict[k] = v

        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

        print(
            'Successfully loaded pretrained %d paras, and %d paras are unmatched.'
            %(len(pretrained_dict.keys()), len(unmatched_pretrained_dict.keys())))
        print('Unmatched pretrained paras are:', unmatched_pretrained_dict.keys())
    return model

def convnext_xlarge(pretrained=True, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048], **kwargs)
    if pretrained:
        assert in_22k, "only ImageNet-22K pre-trained ConvNeXt-XL is available; please set in_22k=True"
        checkpoint = torch.load(kwargs['checkpoint'], map_location="cpu")
        #url = model_urls['convnext_xlarge_22k']
        #checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
        model_dict = model.state_dict()
        pretrained_dict = {}
        unmatched_pretrained_dict = {}

        for k, v in checkpoint['model'].items():
            if k in model_dict:
                pretrained_dict[k] = v
            else:
                unmatched_pretrained_dict[k] = v
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

        print(
            'Successfully loaded pretrained %d paras, and %d paras are unmatched.'
            %(len(pretrained_dict.keys()), len(unmatched_pretrained_dict.keys())))
        print('Unmatched pretrained paras are:', unmatched_pretrained_dict.keys())
    return model

---

# Dataset Definition

In [12]:
# Define a custom dataset class for loading texture and roughness data
class TextureDataset(Dataset):
    def __init__(self, texture_paths, roughness_paths, transform=None):
        """
        Args:
            texture_paths (list): List of paths to texture images.
            roughness_paths (list): List of paths to roughness images.
            transform (callable, optional): Optional transform to be applied
                on both input and target images.
        """
        self.texture_paths = texture_paths
        self.roughness_paths = roughness_paths
        self.transform = transform

    def __len__(self):
        return len(self.texture_paths)

    def __getitem__(self, idx):
        # Load images
        texture = Image.open(self.texture_paths[idx]).convert("RGB")
        roughness = Image.open(self.roughness_paths[idx]).convert("L")

        # Apply transforms if defined
        if self.transform:
            texture = self.transform(texture)
            roughness = self.transform(roughness)


        return {"input": texture, "target": roughness}

---

# Extract Texture and Roughness Paths from CSV Files

In [13]:
# Specify the directory containing your CSV files
csv_directory = "/kaggle/input/texture-allpathds"  # Replace with your directory path

# Initialize lists to store paths
texture_paths = []
roughness_paths = []

# Iterate over all CSV files in the directory
for file_name in os.listdir(csv_directory):
    if file_name.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(csv_directory, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Extract columns and append to the lists
        if "Texture" in df.columns and "Roughness" in df.columns:
            texture_paths.extend(df["Texture"].dropna().tolist())
            roughness_paths.extend(df["Roughness"].dropna().tolist())
        else:
            print(f"Warning: File {file_name} does not contain 'texture' and 'Roughness' columns.")

---

# Dataset Preparation

In [20]:
# Split data into training and testing sets
train_textures, test_textures, train_roughnesses, test_roughnesses = train_test_split(
    texture_paths, roughness_paths, test_size=0.01, random_state=42
)

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((600, 600)),  # Resize images
    transforms.ToTensor(),          # Convert images to tensors
])

# Create training and testing datasets
train_dataset = TextureDataset(train_textures, train_roughnesses, transform=transform)
test_dataset = TextureDataset(test_textures, test_roughnesses, transform=transform)

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

---

# Model Instantiation

In [ ]:
model = convnext_base(True, in_22k=False).cuda()

---

# Training Configuration

In [22]:
# Set optimizer
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

---

# Custom berHu Loss Function

In [23]:
class berHuLoss(nn.Module):
    def __init__(self):
        super(berHuLoss, self).__init__()

    def forward(self, pred, target):
        if pred.shape != target.shape:
            raise ValueError("Predicted and target images must have the same dimensions.")

        error = torch.abs(pred - target)

        c = 0.2 * torch.max(error)

        loss = torch.where(
            error <= c,
            error,
            (error ** 2 + c ** 2) / (2 * c)
        )

        return torch.mean(loss)

In [24]:
# Instantiate the berHuLoss
criterion = berHuLoss()

---

# Logging into Weights & Biases (W&B) Using a User Secret API Key

In [ ]:
# Define a generic user secret

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")
 
#Login to W&B using the retrieved API key
wandb.login(key=secret_value_0)

---

# Saving and Uploading a PyTorch Model to the Hugging Face Hub

In [ ]:
# Set the Hugging Face token from environment variables (ensure it's set in your Kaggle environment)
hf_token = user_secrets.get_secret("HF_TOKEN")


# Log in using the token
login(token=hf_token)

repo_name = "ConvNeXt_roughness_model"
create_repo(repo_name, exist_ok=True)

def save_to_huggingface(model):
    # Save the model to a .pth file
    save_path = "ConvNeXt_roughness_model.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Model saved locally to {save_path}")

    upload_file(
        path_or_fileobj=save_path,
        path_in_repo=save_path,
        repo_id=f"prakanda/{repo_name}",  # Replace with your Hugging Face username
        token=hf_token  # Using the token from environment variable
    )
    print(f"Model uploaded to Hugging Face Hub: https://huggingface.co/prakanda/{repo_name}")

---

# Training Loop

In [ ]:
device0="cuda:0"
num_epochs = 1
wandb.init(project="roughness-estimation-ConvNeXt", config={"epochs": 1, "batch_size": 2, "learning_rate": 1e-4})


# Move the model to the GPU before the training loop
model.to(device0) 


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, batch in enumerate(train_dataloader):
        # Move inputs, targets, and camera_intrinsic to the device
        inputs = batch["input"].to(device0, non_blocking=True)
        targets = batch["target"].to(device0, non_blocking=True)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        predicted_roughness = outputs[-1]

        # Use the berHuLoss criterion for roughness loss calculation
        loss = criterion(predicted_roughness, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Log progress every 10 batches
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch + 1}, Batch {batch_idx}, Loss: {loss.item():.4f}")

        wandb.log({"epoch": epoch + 1, "batch_loss": loss.item()})

        # Save the model every 1000 batches
        if batch_idx != 0 and batch_idx % 1000 == 0:
            print("Saving model")
            save_to_huggingface(model)

    # Log metrics to W&B
    average_loss_per_epoch = running_loss / len(train_dataloader)
    wandb.log({"epoch": epoch + 1, "average_loss_per_epoch": average_loss_per_epoch})

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss_per_epoch:.4f}")

print("Training completed!")

---

# Loading Model from Hugging Face

In [26]:
# Set the Hugging Face token from environment variables (ensure it's set in your Kaggle environment)
hf_token = user_secrets.get_secret("HF_TOKEN")


# Log in using the token
login(token=hf_token)

In [ ]:
# Download the model file from Hugging Face Hub
repo_name = "ConvNeXt_roughness_model"
downloaded_file = hf_hub_download(
    repo_id=f"prakanda/{repo_name}",  # Replace with your Hugging Face username
    filename="ConvNeXt_roughness_model.pth"
)
print(f"Model downloaded from Hugging Face Hub: {downloaded_file}")



# Initialize the model and load the state_dict

model.load_state_dict(torch.load(downloaded_file),strict=False)

---

# Evaluation Loop

In [ ]:
device0="cuda:0"
wandb.init(project="roughness-estimation-ConvNeXt", config={"epochs": 1, "batch_size": 2, "learning_rate": 1e-4})

def evaluate(model, test_dataloader, criterion):
    model.eval()
    running_loss = 0.0  # Accumulate loss over all batches
    total_samples = 0   # Track the number of processed samples

    with torch.no_grad():
        for batch_idx, batch in enumerate(test_dataloader):
            # Move inputs and targets to the device
            inputs = batch["input"].to(device0, non_blocking=True)
            targets = batch["target"].to(device0, non_blocking=True)

            # Forward pass
            outputs = model(inputs)
            predicted_roughness = outputs[-1]

            # Calculate loss
            loss = criterion(predicted_roughness, targets)

            # Accumulate running loss and sample count
            running_loss += loss.item() * inputs.size(0)  # Weighted by batch size
            total_samples += inputs.size(0)

        # Calculate average loss over the dataset
        avg_loss = running_loss / total_samples
        return avg_loss

# Perform evaluation on the test set
test_loss = evaluate(model, test_dataloader, criterion)
print(f"Test Loss: {test_loss:.4f}")

# Log the average test loss to W&B
wandb.log({"average_test_loss": test_loss})

# Finish W&B run
wandb.finish()

print("Evaluation completed!")